# Chapter 3: Setting Up Your Hugging Face Environment

This notebook contains all examples from Chapter 3, demonstrating how to set up and use the Hugging Face ecosystem.

## 1. Environment Verification

First, let's verify that all required packages are installed correctly.

In [1]:
# Verify installations
import transformers
import datasets
import accelerate
import torch
import huggingface_hub

print("Transformers:", transformers.__version__)
print("Datasets:", datasets.__version__)
print("Accelerate:", accelerate.__version__)
print("PyTorch:", torch.__version__)
print("HF Hub:", huggingface_hub.__version__)

# Check device availability
print("\nDevice Information:")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA device: {torch.cuda.get_device_name(0)}")
if hasattr(torch.backends, "mps"):
    print(f"MPS available: {torch.backends.mps.is_available()}")

Transformers: 4.39.3
Datasets: 3.6.0
Accelerate: 1.8.1
PyTorch: 2.7.1
HF Hub: 0.33.2

Device Information:
CUDA available: False
MPS available: True


## 2. Basic Pipeline Example

HuggingFace pipelines provide a simple API for common NLP tasks.

In [2]:
from transformers import pipeline

# Create a sentiment analysis pipeline
classifier = pipeline("sentiment-analysis")

# Test it
results = classifier([
    "I love HuggingFace!",
    "This is terrible.",
    "The weather is okay today."
])

for text, result in zip(["I love HuggingFace!", "This is terrible.", "The weather is okay today."], results):
    print(f'"{text}" -> {result["label"]} (score: {result["score"]:.3f})')

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
/Users/richardhightower/src/art_hug_03/.venv/lib/python3.12/site-packages/huggingface_hub/file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


"I love HuggingFace!" -> POSITIVE (score: 1.000)
"This is terrible." -> NEGATIVE (score: 1.000)
"The weather is okay today." -> POSITIVE (score: 1.000)


## 3. HuggingFace Hub API

Explore models available on the HuggingFace Hub.

In [3]:
from huggingface_hub import HfApi

# Create an API client
api = HfApi()

# List text-classification models
models = list(api.list_models(task="text-classification", limit=100))

print(f"Found {len(models)} text-classification models!")
print("\nTop 5 most downloaded:")

# Show top 5
sorted_models = sorted(models, key=lambda x: x.downloads or 0, reverse=True)[:5]
for i, model in enumerate(sorted_models, 1):
    print(f"{i}. {model.modelId} (Downloads: {model.downloads:,})")

Found 100 text-classification models!

Top 5 most downloaded:
1. cardiffnlp/twitter-roberta-base-sentiment-latest (Downloads: 4,556,383)
2. cross-encoder/ms-marco-MiniLM-L6-v2 (Downloads: 4,336,948)
3. distilbert/distilbert-base-uncased-finetuned-sst-2-english (Downloads: 3,504,257)
4. facebook/bart-large-mnli (Downloads: 2,590,462)
5. BAAI/bge-reranker-v2-m3 (Downloads: 2,363,490)


## 4. Model Download Example

Download and use a specific model with tokenizer.

In [4]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer

model_name = "distilbert-base-uncased-finetuned-sst-2-english"

# Download tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

print(f"Model loaded: {model_name}")
print(f"Model type: {model.config.model_type}")
print(f"Number of parameters: {sum(p.numel() for p in model.parameters()):,}")

Model loaded: distilbert-base-uncased-finetuned-sst-2-english
Model type: distilbert
Number of parameters: 66,955,010


In [5]:
# Use the model for inference
import torch

text = "HuggingFace makes NLP so much easier!"
inputs = tokenizer(text, return_tensors="pt")

with torch.no_grad():
    outputs = model(**inputs)
    predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)
    
print(f"Text: '{text}'")
print(f"Prediction: Negative={predictions[0][0]:.3f}, Positive={predictions[0][1]:.3f}")

Text: 'HuggingFace makes NLP so much easier!'
Prediction: Negative=0.006, Positive=0.994


## 5. Translation Pipeline

Demonstrate translation with batch processing.

In [6]:
# Create translation pipeline
device = 0 if torch.cuda.is_available() else -1

translator = pipeline(
    "translation_en_to_fr",
    model="Helsinki-NLP/opus-mt-en-fr",
    device=device
)

sentences = [
    "Hugging Face makes AI easy.",
    "Transformers are powerful."
]

# Translate with batch processing
translations = translator(sentences, batch_size=2)

for original, result in zip(sentences, translations):
    print(f"EN: {original}")
    print(f"FR: {result['translation_text']}\n")

/Users/richardhightower/src/art_hug_03/.venv/lib/python3.12/site-packages/transformers/models/marian/tokenization_marian.py:197: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


EN: Hugging Face makes AI easy.
FR: Le visage hugging rend l'IA facile.

EN: Transformers are powerful.
FR: Les transformateurs sont puissants.



## 6. Text Generation

Generate text using GPT-2 model.

In [7]:
# Text generation pipeline
generator = pipeline(
    "text-generation",
    model="distilgpt2",
    device=device
)

prompt = "The future of artificial intelligence is"

# Generate text
result = generator(
    prompt,
    max_length=50,
    num_return_sequences=2,
    temperature=0.8
)

print(f"Prompt: '{prompt}'")
print("\nGenerated continuations:")
for i, generated in enumerate(result, 1):
    print(f"\n{i}. {generated['generated_text']}")

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Prompt: 'The future of artificial intelligence is'

Generated continuations:

1. The future of artificial intelligence is not just a matter of time – it's the future of technology."

2. The future of artificial intelligence is uncertain. We’re moving towards AI with a goal of achieving the goal of becoming an artificial intelligence: to create a world where computers aren't like computers. This is not just something the world's leaders have been


## 7. Zero-Shot Classification

Classify text without training on specific labels.

In [8]:
# Zero-shot classification
classifier = pipeline("zero-shot-classification", device=device)

text = "This is a tutorial about natural language processing with transformers."
candidate_labels = ["education", "politics", "entertainment", "technology", "sports"]

result = classifier(text, candidate_labels)

print(f"Text: '{text}'")
print("\nClassification scores:")
for label, score in zip(result['labels'], result['scores']):
    print(f"  {label}: {score:.3f}")

No model was supplied, defaulted to facebook/bart-large-mnli and revision c626438 (https://huggingface.co/facebook/bart-large-mnli).
Using a pipeline without specifying a model name and revision in production is not recommended.


Text: 'This is a tutorial about natural language processing with transformers.'

Classification scores:
  technology: 0.920
  education: 0.051
  entertainment: 0.017
  sports: 0.009
  politics: 0.005


## 8. Question Answering

Extract answers from context using QA models.

In [9]:
# Question answering pipeline
qa_pipeline = pipeline("question-answering", device=device)

context = """
HuggingFace is a company that develops tools for building applications using machine learning.
It is most notable for its Transformers library built for natural language processing applications
and its platform that allows users to share machine learning models and datasets.
The company was founded in 2016 by Clément Delangue, Julien Chaumond, and Thomas Wolf.
"""

questions = [
    "What is HuggingFace?",
    "When was the company founded?",
    "Who founded HuggingFace?"
]

for question in questions:
    result = qa_pipeline(question=question, context=context)
    print(f"Q: {question}")
    print(f"A: {result['answer']} (score: {result['score']:.3f})\n")

No model was supplied, defaulted to distilbert/distilbert-base-cased-distilled-squad and revision 626af31 (https://huggingface.co/distilbert/distilbert-base-cased-distilled-squad).
Using a pipeline without specifying a model name and revision in production is not recommended.


config.json:   0%|          | 0.00/473 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/261M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

Q: What is HuggingFace?
A: a company that develops tools for building applications using machine learning (score: 0.602)

Q: When was the company founded?
A: 2016 (score: 0.982)

Q: Who founded HuggingFace?
A: Clément Delangue, Julien Chaumond, and Thomas Wolf (score: 0.842)



## 9. Named Entity Recognition

Identify entities in text.

In [10]:
# NER pipeline
ner = pipeline("ner", aggregation_strategy="simple", device=device)

text = "Apple Inc. was founded by Steve Jobs in Cupertino, California. The company is now led by Tim Cook."

entities = ner(text)

print(f"Text: '{text}'")
print("\nEntities found:")
for entity in entities:
    print(f"  {entity['word']} -> {entity['entity_group']} (score: {entity['score']:.3f})")

No model was supplied, defaulted to dbmdz/bert-large-cased-finetuned-conll03-english and revision f2482bf (https://huggingface.co/dbmdz/bert-large-cased-finetuned-conll03-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


config.json:   0%|          | 0.00/998 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.33G [00:00<?, ?B/s]

Some weights of the model checkpoint at dbmdz/bert-large-cased-finetuned-conll03-english were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


tokenizer_config.json:   0%|          | 0.00/60.0 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

Text: 'Apple Inc. was founded by Steve Jobs in Cupertino, California. The company is now led by Tim Cook.'

Entities found:
  Apple Inc -> ORG (score: 1.000)
  Steve Jobs -> PER (score: 0.993)
  Cupertino -> LOC (score: 0.977)
  California -> LOC (score: 0.999)
  Tim Cook -> PER (score: 1.000)


## 10. Model Comparison

Compare different models for the same task.

In [11]:
# Compare sentiment analysis models
models = [
    "distilbert-base-uncased-finetuned-sst-2-english",
    "nlptown/bert-base-multilingual-uncased-sentiment"
]

text = "This product is amazing! I highly recommend it."

for model_name in models:
    try:
        classifier = pipeline("sentiment-analysis", model=model_name, device=device)
        result = classifier(text)
        print(f"Model: {model_name}")
        print(f"Result: {result[0]}\n")
    except Exception as e:
        print(f"Model: {model_name}")
        print(f"Error: {str(e)[:100]}...\n")

Model: distilbert-base-uncased-finetuned-sst-2-english
Result: {'label': 'POSITIVE', 'score': 0.9998874664306641}



config.json:   0%|          | 0.00/953 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/669M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/39.0 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

Model: nlptown/bert-base-multilingual-uncased-sentiment
Result: {'label': '5 stars', 'score': 0.9384052157402039}



## 11. Batch Processing Performance

Demonstrate the performance benefits of batch processing.

In [12]:
import time

# Create sentiment analysis pipeline
classifier = pipeline("sentiment-analysis", device=device)

# Test texts
texts = [
    "I love this!",
    "This is terrible.",
    "Not bad at all.",
    "Could be better.",
    "Absolutely fantastic!",
    "Waste of time.",
    "Pretty good overall.",
    "Highly disappointed."
]

# Single processing
start = time.time()
single_results = []
for text in texts:
    result = classifier(text)
    single_results.append(result)
single_time = time.time() - start

# Batch processing
start = time.time()
batch_results = classifier(texts, batch_size=4)
batch_time = time.time() - start

print(f"Single processing time: {single_time:.3f}s")
print(f"Batch processing time: {batch_time:.3f}s")
print(f"Speedup: {single_time/batch_time:.2f}x")

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


Single processing time: 0.157s
Batch processing time: 0.032s
Speedup: 4.90x


## 12. Cache Information

Check the HuggingFace cache directory.

In [13]:
import os
from pathlib import Path

# Get cache directory
cache_dir = Path.home() / ".cache" / "huggingface" / "hub"

if cache_dir.exists():
    # Count cached models
    model_dirs = [d for d in cache_dir.iterdir() if d.is_dir()]
    
    # Calculate total size
    total_size = sum(f.stat().st_size for f in cache_dir.rglob('*') if f.is_file())
    
    print(f"Cache directory: {cache_dir}")
    print(f"Number of cached models: {len(model_dirs)}")
    print(f"Total cache size: {total_size / (1024**3):.2f} GB")
else:
    print("No cache directory found yet.")

Cache directory: /Users/richardhightower/.cache/huggingface/hub
Number of cached models: 22
Total cache size: 19.59 GB


## 13. HuggingFace Spaces Example

Example code for deploying to HuggingFace Spaces with Gradio.

In [14]:
# Example Gradio app for HuggingFace Spaces
# Note: This is example code - Gradio needs to be installed separately

example_gradio_code = '''
import gradio as gr
from transformers import pipeline

# Initialize pipeline
classifier = pipeline("sentiment-analysis")

def analyze_sentiment(text):
    results = classifier(text)
    return {
        "label": results[0]["label"],
        "score": results[0]["score"]
    }

# Create Gradio interface
iface = gr.Interface(
    fn=analyze_sentiment,
    inputs=gr.Textbox(lines=3, placeholder="Enter text to analyze..."),
    outputs=gr.JSON(),
    title="Sentiment Analysis Demo",
    description="Analyze the sentiment of your text using HuggingFace Transformers",
    examples=[
        ["I love this product!"],
        ["This is terrible."],
        ["It's okay, nothing special."]
    ]
)

# Launch the app
iface.launch()
'''

print("Example Gradio app for HuggingFace Spaces:")
print("=" * 50)
print(example_gradio_code)

Example Gradio app for HuggingFace Spaces:

import gradio as gr
from transformers import pipeline

# Initialize pipeline
classifier = pipeline("sentiment-analysis")

def analyze_sentiment(text):
    results = classifier(text)
    return {
        "label": results[0]["label"],
        "score": results[0]["score"]
    }

# Create Gradio interface
iface = gr.Interface(
    fn=analyze_sentiment,
    inputs=gr.Textbox(lines=3, placeholder="Enter text to analyze..."),
    outputs=gr.JSON(),
    title="Sentiment Analysis Demo",
    description="Analyze the sentiment of your text using HuggingFace Transformers",
    examples=[
        ["I love this product!"],
        ["This is terrible."],
        ["It's okay, nothing special."]
    ]
)

# Launch the app
iface.launch()



## Summary

This notebook covered the essential components of setting up and using the HuggingFace ecosystem:

1. **Environment Verification** - Checking installations
2. **Pipelines** - Simple API for common NLP tasks
3. **Hub API** - Discovering and exploring models
4. **Model Loading** - Downloading and using specific models
5. **Various NLP Tasks** - Translation, generation, QA, NER, etc.
6. **Performance** - Batch processing benefits
7. **Deployment** - Example for HuggingFace Spaces

These examples provide a solid foundation for working with HuggingFace Transformers!